In [880]:
from datetime import datetime
from typing import Iterable, Tuple

from pptx import Presentation


In [ ]:
ppt_template_path = "./myPPT_template.pptx"

In [882]:
# Helpers
def read_pptx(filename):
    """
    Read a PowerPoint presentation (.pptx) file and return the presentation object.

    Parameters:
    - filename (str): The name of the PowerPoint (.pptx) file to read.

    Returns:
    - prs (Presentation): The presentation object.
    """
    # Create a presentation object
    prs = Presentation(filename)

    return prs


def get_layout_mapping(prs) -> dict:
    """
    This function returns a dictionary mapping slide layout names to their indices.

    Parameters:
    - prs (Presentation): The presentation object to extract slide layouts from.

    Returns:
    - lay2idx (dict): A dictionary where the keys are the names of slide layouts and the values are their indices in the presentation.
    """
    # Mapping between slide layout names and indices
    lay2idx = {slide_layout.name: i for i, slide_layout in enumerate(prs.slide_layouts)}
    return lay2idx


def get_slide_and_shapes(
    name: str, prs: "Presentation", lay2idx: dict
) -> Tuple["Presentation", dict]:
    """
    This function gets a slide given a text and shapes within it.

    Parameters:
    prs (Presentation): The PowerPoint presentation object.
    text (str): The text to search for in the presentation.

    Returns:
    slide (Slide): The slide object containing the text.
    """
    # Target slide id
    layout_id = lay2idx[name]
    # Target slide layout
    layout = prs.slide_layouts[layout_id]
    # Target slide
    slide = prs.slides.add_slide(layout)
    # Mapping between target slide shapes and indices
    shape2idx = {s.name: i for i, s in enumerate(slide.shapes)}
    return slide, shape2idx


def add_title(title: str, slide, title_id: int) -> None:
    """
    Add a title to a given slide.

    Parameters:
    - slide (pptx.slides.Slide): The slide to which the title will be added.
    - title (str): The text of the title.

    Returns:
    - None: This function does not return any value. It modifies the slide object.
    """
    title_shape = slide.shapes[title_id]
    title_shape.text = title


def set_title_slide(
    prs: "Presentation", title: str, subtitle: str, date: str
) -> "Presentation":
    """
    This function sets the title slide of a PowerPoint presentation.

    Parameters:
    prs (Presentation): The presentation object to which the title slide will be added.
    title (str): The main title of the presentation.
    subtitle (str): The subtitle of the presentation.
    date (str): The date to be displayed on the title slide.

    Returns:
    prs (Presentation): The updated presentation object with the added title slide.

    Note:
    - The function assumes that the presentation template has a title slide layout at index 0.
    - The function accesses the shapes of the title slide by their index. This may not work if the
      placeholder names from the layout are reset when the actual slide is instantiated.
    """
    # Title slide layout
    title_slide_layout = prs.slide_layouts[0]

    # Add a title slide based on the title slide layout
    title_slide = prs.slides.add_slide(title_slide_layout)

    # TODO: Placeholder names from layout seem to be reset
    # TODO: when actual slide is instantiated
    # Accessing shapes by index (assuming placeholders are not renamed)
    title_shape0 = title_slide.shapes[0]
    title_shape1 = title_slide.shapes[1]
    title_shape2 = title_slide.shapes[2]

    # Set the text for each shape
    title_shape0.text = date
    title_shape1.text = title
    title_shape2.text = subtitle

    return prs


def set_toc(prs, toc_id, toc_title, toc_content) -> None:
    """
    This function sets the table of contents (TOC) slide in a PowerPoint presentation.

    Parameters:
    prs (Presentation): The presentation object to which the TOC slide will be added.
    toc_id (int): The index of the slide layout for the TOC slide.
    toc_title (str): The title of the TOC slide.
    toc_content (list[str]): A list of strings representing the content of the TOC.

    Returns:
    prs (Presentation): The updated presentation object with the added TOC slide.

    Note:
    - The function assumes that the presentation template has a slide layout for the TOC slide.
    - The function accesses the shapes of the TOC slide by their index. This may not work if the
      placeholder names from the layout are reset when the actual slide is instantiated.
    """
    # TOC layout
    toc_layout = prs.slide_layouts[toc_id]

    # Create & add TOC slide
    toc_slide = prs.slides.add_slide(toc_layout)

    # TODO: move somewhere else
    # Mapping between slide shapes and indices
    shape2idx = {s.name: i for i, s in enumerate(toc_slide.shapes)}

    # Add title to TOC slide
    toc_title_id = shape2idx["Title 2"]
    add_title(title=toc_title, slide=toc_slide, title_id=toc_title_id)

    # Populate table of contents with section names
    toc_content_id = shape2idx["Text Placeholder 1"]
    toc_content_shape = toc_slide.shapes[toc_content_id]

    toc_text_frame = toc_content_shape.text_frame

    # Populate table of contents with section names
    p = toc_text_frame.paragraphs[0]
    p.text = toc_content[0]

    for para_str in toc_content[1:]:
        p = toc_text_frame.add_paragraph()
        p.text = para_str

    return prs


# def add_image_and_text_slide(prs, image_path, title, content):
#     """
#     This function adds a new slide to a PowerPoint presentation with an image on the left and text on the right.

#     Parameters:
#     prs (Presentation): The presentation object to which the new slide will be added.
#     image_path (str): The path to the image file to be added to the slide.
#     title (str): The title of the slide.
#     content (str): The text content to be added to the slide.

#     Returns:
#     None: This function does not return any value. It modifies the presentation object.

#     Note:
#     - This function assumes that the presentation template has a slide layout with a picture placeholder.
#     - The function uses the pptx library to manipulate the PowerPoint presentation.
#     """

#     # Choose a slide layout with content and picture placeholder
#     slide_layout = prs.slide_layouts[
#         5
#     ]  # 5 corresponds to 'Title Slide with Picture Placeholder'

#     # Add the new slide to the presentation
#     slide = prs.slides.add_slide(slide_layout)

#     # Set title
#     title_shape = slide.shapes.title
#     title_shape.text = title

#     # Add image on the left side
#     left_inch = Inches(0.5)
#     top_inch = Inches(1.5)
#     pic = slide.shapes.add_picture(image_path, left_inch, top_inch, width=Inches(4))

#     # Add text on the right side
#     text_box = slide.shapes.add_textbox(Inches(5), top_inch, Inches(3), Inches(5))
#     text_frame = text_box.text_frame
#     p = text_frame.add_paragraph()
#     p.text = content
#     p.alignment = PP_ALIGN.LEFT
#     p.font.size = Pt(18)


def set_bullet_points(bp_dict: Iterable, text_frame: "Text Placeholder") -> None:
    """
    This function populates a text frame with bullet points based on a given dictionary.

    Parameters:
    bp_dict (Iterable): A dictionary containing bullet point keys and values.
    text_frame (TextFrame): The text frame where the bullet points will be added.

    Returns:
    None: This function does not return any value. It modifies the text frame directly.
    """
    for i, (k, v) in enumerate(bp_dict.items()):
        if i == 0:
            p = text_frame.paragraphs[0]
        else:
            p = text_frame.add_paragraph()
        p.level = 0

        # Add dataset feature in bold
        run = p.add_run()
        run.text = f"{k}: "
        run.font.bold = True

        # Add dataset feature content
        if isinstance(v, str):
            run = p.add_run()
            run.text = v
        elif isinstance(v, list):
            for i, item in enumerate(v):
                p = text_frame.add_paragraph()
                p.text = item
                p.level = 1


def set_img(
    slide: "Slide", img_path: str, placedholder_name: str, shape2idx: dict
) -> None:
    """
    This function inserts an image into a PowerPoint image placeholder.

    Parameters:
    - slide (Slide): The slide to which the image will be added.
    - img_path (str): The path to the image file to be added to the slide.
    - placedholder_name (str): The name of the image placeholder in the slide layout.
    - shape2idx (dict): A dictionary mapping shape names to their indices in the slide.

    Returns:
    None: This function does not return any value. It modifies the slide object directly.

    Note:
    - This function assumes that the presentation template has an image placeholder with the given name.
    - The function uses the pptx library to manipulate the PowerPoint presentation.
    """
    pp_id = shape2idx[placedholder_name]
    image_shape = slide.shapes[pp_id]
    img = image_shape.insert_picture(img_path)


def get_text_frame(slide, placeholder_name, shape2idx):
    """
    This function returns the text frame of a PowerPoint placeholder.

    Parameters:
    - slide (pptx.slides.Slide): The slide containing the specified placeholder.
    - placeholder_name (str): The name of the text placeholder within the slide.
    - shape2idx (dict): A dictionary mapping shape names to their indices in the slide.

    Returns:
    - text_frame (pptx.slides.shapes.TextPlaceholder): The text frame of the specified placeholder.

    This function retrieves the text frame associated with a given text placeholder within a slide. It uses the provided dictionary `shape2idx` to map the names of the slide shapes to their indices, allowing for efficient retrieval of the desired shape.

    Example usage:

    ```python
    slide = prs.slides[0]  # Assuming prs is the presentation object
    placeholder_name = "Text Placeholder 1"
    shape2idx = {s.name: i for i, s in enumerate(slide.shapes)}
    text_frame = get_text_frame(slide, placeholder_name, shape2idx)
    ```
    """
    pp_id = shape2idx[placeholder_name]
    text_shape = slide.shapes[pp_id]
    text_frame = text_shape.text_frame
    return text_frame

[Modeling the spatial dependence of floods](https://hess.copernicus.org/articles/23/107/2019/hess-23-107-2019.pdf)

# Read powerpoint template file

In [883]:
# Load PPT template
prs = read_pptx(ppt_template_path)
# Map PPT template slides to indices for convenience
lay2idx = get_layout_mapping(prs=prs)

In [884]:
lay2idx

{'Titre': 0,
 'Section': 1,
 'Goals': 2,
 'TOC': 3,
 'Bullet points + Image': 4,
 'Contenu classique': 5,
 'Contenu classique avec note': 6,
 '2 contenus': 7,
 '2 contenus + couleur droite': 8,
 '1_2 contenus + 2 couleurs': 9,
 '2 contenus + 2 couleurs': 10,
 'Graphe et commentaire': 11,
 '2 Graphes et commentaires': 12,
 '2 Graphes et 1 commentaire': 13,
 'Titre seul': 14,
 'Conclusion': 15,
 'Annexe': 16}

## Title slide

In [885]:
# Presentation title
TITLE = "Modelling flood-related physical risks"
# Presentation subtitle
SUBTITLE = (
    "The importance of integrating spatial dependence for reliable damage estimates"
)
# Get the current date
current_date = datetime.now()
# Format the date as DD/MM/YYYY
formatted_date = current_date.strftime("%d/%m/%Y")

In [886]:
set_title_slide(prs=prs, title=TITLE, subtitle=SUBTITLE, date=formatted_date)

# Objectives/Motivation slide

In [887]:
# "Goals" slide
goals_id = lay2idx["Goals"]
# "Goals" slide layout
goals_layout = prs.slide_layouts[goals_id]
# Instantiate "Goals" slide & add to presentation
goals_slide = prs.slides.add_slide(goals_layout)
# Mapping between slide shapes and indices
shape2idx = {s.name: i for i, s in enumerate(goals_slide.shapes)}
# Add slide title
title_id = shape2idx["Title 1"]
title = (
    "We investigate the impact of future flood hazards on existing physical assets, \
thanks to some of the most recent global and open-source datasets"
)
add_title(title=title, slide=goals_slide, title_id=title_id)

# Add text to text placedholders
text_placeholders = [
    "Text Placeholder 3",
    "Text Placeholder 2",
    "Text Placeholder 6",
    "Text Placeholder 8",
]
# Input: goals, challenges, ..., ...
contents = {
    "Goal": "estimating the impact of flood hazards on existing physical assets",
    "Challenges": "current climate intensity projections do not account for the interdependence between hazards",
    "Approach": "we resort to Extreme Value Theory, and copulas in particular, to model the dependence between extreme flood events",
}

contents_list = list(contents.items())

for i, text_placeholder in enumerate(text_placeholders):
    text_id = shape2idx[text_placeholder]
    text_shape = goals_slide.shapes[text_id]
    text_frame = text_shape.text_frame
    # Populate table of contents with section names
    p = text_frame.paragraphs[0]
    try:
        k, v = contents_list[i]
    except IndexError:
        k, v = "", ""
    p.text = f"{k}: {v}"

# Icons
goals_logo_path = (
    "C:/Users/hamada.saleh/Documents/Python Scripts/AutoPPT/flood_pres/goal_logo.png"
)
challenges_logo_path = "C:/Users/hamada.saleh/Documents/Python Scripts/AutoPPT/flood_pres/challenges_logo.png"
solution_logo_path = "C:/Users/hamada.saleh/Documents/Python Scripts/AutoPPT/flood_pres/solution_logo.png"
set_img(
    slide=goals_slide,
    img_path=goals_logo_path,
    placedholder_name="Picture Placeholder 4",
    shape2idx=shape2idx,
)
set_img(
    slide=goals_slide,
    img_path=challenges_logo_path,
    placedholder_name="Picture Placeholder 5",
    shape2idx=shape2idx,
)
set_img(
    slide=goals_slide,
    img_path=solution_logo_path,
    placedholder_name="Picture Placeholder 7",
    shape2idx=shape2idx,
)

In [888]:
shape2idx

{'Title 1': 0,
 'Text Placeholder 2': 1,
 'Text Placeholder 3': 2,
 'Picture Placeholder 4': 3,
 'Picture Placeholder 5': 4,
 'Text Placeholder 6': 5,
 'Picture Placeholder 7': 6,
 'Text Placeholder 8': 7}

# Outline/TOC slide

In [889]:
# Slide "Sommaire"
toc_id = lay2idx["TOC"]
# TOC title
toc_title = "Table of contents"

toc_content = [
    "Datasets 0. Physical assets: Climate Trace 1. Flood hazards: Global Flood Database",
    "Extreme value theory: modelling tail dependencies between flood events. 0. Intuition behind flood modelling 1. Theoretical framework 2. Application",
    "Results",
    "Conclusion & Next steps",
]
set_toc(prs=prs, toc_id=toc_id, toc_title=toc_title, toc_content=toc_content)

# Add bullets points + Image slide

## Dataset presentation slide

In [890]:
# Get "Bullet points + Image" slide
name = "Bullet points + Image"
content_slide, shape2idx = get_slide_and_shapes(name=name, prs=prs, lay2idx=lay2idx)

0. Dataset title

In [891]:
slide_title_id = shape2idx["Title 3"]
# Populate slide title
slide_title = "The Global Flood Database provides 912 historical floods between 2000 and 2018 and allows to estimate the dependence between flood events from regional river networks"
slide_title_shape = content_slide.shapes[slide_title_id]
slide_title_shape.text = slide_title

1. Dataset features

In [892]:
feat_keys = ["Sample size", "Geographic coverage", "Time coverage", "Features"]

In [893]:
shape2idx

{'Text Placeholder 1': 0,
 'Text Placeholder 2': 1,
 'Title 3': 2,
 'Picture Placeholder 4': 3,
 'Picture Placeholder 5': 4}

In [894]:
# Set "Bullet points" icon
icon_path = "C:/Users/hamada.saleh/Documents/Python Scripts/AutoPPT/flood_pres/global_flood_database_logo.png"
set_img(
    slide=content_slide,
    img_path=icon_path,
    placedholder_name="Picture Placeholder 5",
    shape2idx=shape2idx,
)

# Set "Bullet points" title
bp_title_id = shape2idx["Text Placeholder 2"]
bp_title_shape = content_slide.shapes[bp_title_id]
# Populate title shape with title
title = "Global Flood Database"
db_source = "https://global-flood-database.cloudtostreet.ai/#interactive-map"
p = bp_title_shape.text_frame.paragraphs[0]
r = p.add_run()
r.text = title
# Add a hyperlink to the text
hlink = r.hyperlink
hlink.address = db_source

# Set "Bullet points" content

In [895]:
global_flood_database_bp = {
    "Sample size": "913 events",
    "Geographic coverage": "169 countries",
    "Time coverage": "2000 to 2018",
    "Main features": [
        "Provides spatial flood extent, duration, and magnitude.",
        "Offers historic maps to inform current and future vulnerability hot spots.",
    ],
}


In [896]:
text_frame = get_text_frame(
    slide=content_slide, placeholder_name="Text Placeholder 1", shape2idx=shape2idx
)
set_bullet_points(bp_dict=global_flood_database_bp, text_frame=text_frame)


2. Image

In [897]:
shape2idx

{'Text Placeholder 1': 0,
 'Text Placeholder 2': 1,
 'Title 3': 2,
 'Picture Placeholder 4': 3,
 'Picture Placeholder 5': 4}

In [898]:
# Example image of flood in Pakistan from Global Flood Database
img_path = "C:/Users/hamada.saleh/Documents/Python Scripts/AutoPPT/flood_pres/global_flood_database_pakistan_example.png"
set_img(
    slide=content_slide,
    img_path=img_path,
    placedholder_name="Picture Placeholder 4",
    shape2idx=shape2idx,
)

# Conclusion/Next steps slide

In [899]:
# "Conclusion" slide id
ccl_id = lay2idx["Conclusion"]
# "Conclusion" slide layout
ccl_layout = prs.slide_layouts[ccl_id]
# "Conclusion" slide
ccl_slide = prs.slides.add_slide(ccl_layout)
# Mapping between slide shapes and indices
shape2idx = {s.name: i for i, s in enumerate(ccl_slide.shapes)}
# Add slide title
title_id = shape2idx["Title 1"]
ccl_title = "We develop a method for assessing the impact of flood hazards under future climate conditions and taking regional dependencies between river catchments into account."
add_title(title=ccl_title, slide=ccl_slide, title_id=title_id)
# Add text to text placedholders
text_placeholders = [
    "Text Placeholder 2",  # "Next Steps"
    "Text Placeholder 3",  # "Main Conclusions"
    "Text Placeholder 4",  # "Main Achievements"
]

# Bullet points list
values = [
    ["How do physical impacts translate into financial impacts?"],
    [""],
    [
        "Module for matching large-scale asset-level datasets with global hazard maps.",
        "Copula-based module for capturing tail dependencies between neighbouring river catchments.",
    ],
]
for i, text_placeholder in enumerate(text_placeholders):
    text_id = shape2idx[text_placeholder]
    text_shape = ccl_slide.shapes[text_id]
    text_frame = text_shape.text_frame
    # content
    v = values[i]
    # Populate table of contents with section names
    p = text_frame.paragraphs[0]
    p.text = v[0]
    p.level = 0
    if len(v) > 1:
        for j in range(1, len(v)):
            p = text_frame.add_paragraph()
            p.text = v[j]
            p.level = 0

In [900]:
shape2idx

{'Title 1': 0,
 'Text Placeholder 2': 1,
 'Text Placeholder 3': 2,
 'Text Placeholder 4': 3}

Set bullet points

In [901]:
prs.save("pladifes.pptx")

# NEW SECTION

In [ ]:
import re
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
from pptx.oxml import parse_xml
from pptx.oxml.ns import nsdecls

In [ ]:
# CONSTANTS
SLD_LAYOUT_TITLE = 

In [ ]:

def parse_md_file(md_file):
    """Parses a markdown file and extracts slide titles and bullet points without labels."""
    slides = []
    with open(md_file, "r", encoding="utf-8") as file:
        content = file.read()

    # Split slides using Markdown headers (e.g., # Slide 1: Title)
    slide_sections = re.split(r"^# (.*?)$", content, flags=re.MULTILINE)[1:]

    for i in range(0, len(slide_sections), 2):
        title = re.sub(r"^(Title Slide:|Slide \d+:|Conclusion Slide:|References Slide:|Appendix Slide:)\s*", "", slide_sections[i].strip(), flags=re.IGNORECASE)  # Remove labels
        bullets = [line.strip("- ").strip() for line in slide_sections[i + 1].split("\n") if line.startswith("-")]
        slides.append((title, bullets))

    return slides

def set_text_format(text_frame, font_name="Arial", font_size=24, color=(0, 0, 0), align=PP_ALIGN.LEFT):
    """Applies font formatting and alignment to a text frame."""
    for paragraph in text_frame.paragraphs:
        paragraph.alignment = align  # Set text alignment (JUSTIFY used in content slides)
        run = paragraph.runs[0] if paragraph.runs else paragraph.add_run()
        run.font.name = font_name
        run.font.size = Pt(font_size)
        run.font.color.rgb = RGBColor(*color)  # Set font color

def parse_hyperlinked_text(text):
    """Extracts (text, URL) pairs from Markdown-style [text](url) hyperlinks."""
    pattern = r"\[(.*?)\]\((.*?)\)"
    return re.findall(pattern, text)

def add_hyperlinked_text(paragraph, text, url):
    """Adds only the hyperlinked text to a paragraph in PowerPoint (without raw MD syntax)."""
    paragraph.clear()  # Remove any default text
    run = paragraph.add_run()
    run.text = text  # Display only the text, not the URL
    run.font.color.rgb = RGBColor(0, 0, 255)  # Blue color for hyperlink
    run.font.size = Pt(20)
    run.hyperlink.address = url  # Make it clickable

In [ ]:
slides = parse_md_file(md_file="slides.md")

In [ ]:
slides

[('Biodiversity Presentation', []),
 ('Introduction to Biodiversity',
  ['Biodiversity refers to the variety of life, including species, ecosystems, and genetic diversity.',
   'It maintains ecological balance and supports essential ecosystem services.',
   'Found in terrestrial, freshwater, and marine environments worldwide.']),
 ('Importance of Biodiversity',
  ['Supports ecosystem stability and resilience to environmental changes.',
   'Provides essential resources like food, medicine, and raw materials.',
   'Contributes to climate regulation, water purification, and soil fertility.']),
 ('Threats to Biodiversity',
  ['Habitat destruction from deforestation, urbanization, and agriculture.',
   'Climate change causing species extinction and ecosystem shifts.',
   'Pollution harming air, water, and soil quality, affecting biodiversity.']),
 ('Conclusion',
  ['Biodiversity is essential for ecosystem stability and human well-being.',
   'Conservation efforts are crucial to protect spec

In [ ]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
from datetime import date


def create_ppt_from_md(md_file, output_pptx, template_pptx = None):
    """Creates a PowerPoint file from a markdown file with justified bullet points and a distinct title slide."""
    slides_content = parse_md_file(md_file)
    if template_pptx is not None:
        prs = Presentation(template_pptx)
    else:
        prs = Presentation()
        
    slide_titles = []  # Store content slide titles for the outline
    appendix_titles = []
    
    for i, (title, bullet_points) in enumerate(slides_content):
        if i == 0 and not bullet_points:
            # Title Slide
            slide = prs.slides.add_slide(prs.slide_layouts[0])
            title_placeholder = slide.shapes.title
            subtitle_placeholder = slide.placeholders[1]

            title_placeholder.text = title
            subtitle_placeholder.text = ""

            # Add date at the bottom-right corner
            left, top = Inches(8.5), Inches(6.9)
            date_textbox = slide.shapes.add_textbox(left, top, Inches(1), Inches(0.5))
            date_text_frame = date_textbox.text_frame
            date_text_frame.text = date.today().strftime("%d-%m-%Y")

            # Apply font formatting
            set_text_format(title_placeholder.text_frame, font_name="Calibri", font_size=44, color=(0, 0, 128))
            set_text_format(subtitle_placeholder.text_frame, font_name="Calibri", font_size=24, color=(80, 80, 80))

        else:
            # Check if slide is an appendix slide
            if title.lower().startswith("appendix"):
                appendix_titles.append(title)
            else:
                slide_titles.append(title)
            # Content Slides
            slide = prs.slides.add_slide(prs.slide_layouts[1])
            title_placeholder = slide.shapes.title
            content_placeholder = slide.placeholders[1]

            title_placeholder.text = title
            content_placeholder.text = ""  # Clear placeholder text

            # Add bullet points with justified alignment
            for point in bullet_points:
                paragraph = content_placeholder.text_frame.add_paragraph()
                paragraph.text = f"{point}"
                paragraph.alignment = PP_ALIGN.JUSTIFY  # Justify text
                paragraph.space_after = Pt(8)
                
                hyperlinks = parse_hyperlinked_text(point)
                if hyperlinks:
                    for text, url in hyperlinks:
                        add_hyperlinked_text(paragraph, text, url)  # Add clickable hyperlink
                else:
                    paragraph.text = point  # Regular text

                # Apply font formatting
                run = paragraph.runs[0]
                run.font.name = "Arial"
                run.font.size = Pt(20)
                run.font.color.rgb = RGBColor(50, 50, 50)  # Dark Gray

            # Apply formatting to title
            set_text_format(title_placeholder.text_frame, font_name="Arial", font_size=32, color=(0, 0, 0))

            # Add slide number (bottom-right corner)
            left, top = Inches(8.5), Inches(6.9)
            slide_number = slide.shapes.add_textbox(left, top, Inches(1), Inches(0.5))
            slide_number.text_frame.text = f"{i}/{len(slides_content) - 1}"
            set_text_format(slide_number.text_frame, font_name="Arial", font_size=14, color=(100, 100, 100), align=PP_ALIGN.RIGHT)
    
    # Create an Outline Slide after the Title Slide
    outline_slide = prs.slides.add_slide(prs.slide_layouts[1])  # Title & Content Layout
    outline_slide.shapes.title.text = "Outline"
    outline_text_frame = outline_slide.placeholders[1].text_frame
    outline_text_frame.clear()

    for slide_title in slide_titles:
        paragraph = outline_text_frame.add_paragraph()
        paragraph.text = slide_title  # Add slide title as bullet point
        paragraph.font.size = Pt(20)
        paragraph.font.color.rgb = RGBColor(0, 0, 0)  # Black
    
    if appendix_titles:
        paragraph = outline_text_frame.add_paragraph()
        paragraph.text = "Appendices"  # Add slide title as bullet point
        paragraph.font.size = Pt(20)
        paragraph.font.color.rgb = RGBColor(0, 0, 0)  # Black
    
    # Move Outline Slide to position 2 (after the Title Slide)
    xml_slides = prs.slides._sldIdLst  # Access slide list
    xml_slides.insert(1, xml_slides[-1])  # Move last slide to index 1 (second position)
    
    prs.save(output_pptx)
    print(f"PowerPoint file '{output_pptx}' has been created successfully with justified bullet points!")


In [ ]:

# Example usage
md_filename = "slides.md"  # Your Markdown file
pptx_filename = "biodiversity.pptx"  # Output PowerPoint file
pptx_template = "myPPT_template.pptx"

create_ppt_from_md(md_filename, pptx_filename, pptx_template)


KeyError: 'no placeholder on this slide with idx == 1'

In [ ]:
prs_def =  Presentation()

In [ ]:
prs_cust = Presentation(pptx_template)